In [1]:
import calendar
import json
import math
import numpy
import os
import pandas
import pyarrow
import sys

from datetime import date, datetime
from os import path
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Steps to install
# 1. pip install sqlalchemy-bigquery google-cloud-bigquery-storage pyarrow
# 2. Copy the credentials file to wherever you set BIGQUERY_CREDENTIALS_PATH to

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

In [2]:
sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
import create_queries
%load_ext autoreload
%autoreload 2

In [21]:
COMPANY_IDENTIFIER = 'SO'
COMPANY_NAME = COMPANY_IDENTIFIER
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'
ANALYSIS_PARAMS = {
    'sold_threshold': 1.0,
    'find_parent_child_relationships': False,
    'use_prices_to_fill_missing_incoming': True,
    'external_pricing_data_config': {
        'category_to_fixed_prices': {
           "Capsule (weight - each)":{
              "Each":23.594097452934655
           },
           "Clone - Cutting":{
              "Each":2.153121902874133
           },
           "Clone - Tissue Culture":{
              "Each": 4.0
           },
           "Edible (volume - each)":{
              "Each":6.146205207527713
           },
           "Edible (weight - each)":{
              "Each":7.5740532898741435
           },
           "Extract (volume - each)":{
              "Each":11.43294776119403
           },
           "Extract (weight - each)":{
              "Each":9.832834338863785
           },
           "Extract (weight)":{
              "Grams":23.60053333333333,
              "Pounds":10705.013115733333
           },
           "Flower":{
              "Grams":0.6615655299919327,
              "Pounds": 0.2
           },
           "Flower (packaged - each)":{
              "Each":6.62190019193858
           },
           "Flower (packaged eighth - each)":{
              "Each":7.620680869582421
           },
           "Flower (packaged gram - each)":{
              "Each":0.36474099339160093
           },
           "Flower (packaged half ounce - each)":{
              "Each":17.340007024938533
           },
           "Flower (packaged ounce - each)":{
              "Each":72.03456896551724
           },
           "Flower (packaged quarter - each)":{
              "Each":14.970933920704848
           },
           "Fresh Cannabis Plant":{
              "Pounds": 4.0
           },
           "Immature Plant":{
              "Each":0.6451699946033459
           },
           "Leaf":{
              "Grams": 4.0,
              "Pounds": 4.0
           },
           "Other Concentrate (volume - each)":{
              "Each":10.0
           },
           "Other Concentrate (weight - each)":{
              "Each": 9.10354624425141
           },
           "Other Concentrate (weight)":{
              "Grams": 4.0
           },
           "Pre-Roll Flower":{
              "Each":1.2455980167454037
           },
           "Pre-Roll Infused":{
              "Each":3.4070715249662618
           },
           "Pre-Roll Leaf":{
              "Each":0.2721679544530297
           },
           "Seeds":{
              "Grams":4.0,
              "Ounces":4.0,
              "Pounds":4.0
           },
           "Seeds (each)":{
              "Each":13.81578947368421
           },
           "Shake":{
              "Pounds": 4.0
           },
           "Shake (Packaged Half Ounce - each)":{
              "Each":13.5
           },
           "Shake (Packaged Quarter - each)":{
              "Each":0.01
           },
           "Tincture (volume - each)":{
              "Each":18.740093427835063
           },
           "Topical (volume - each)":{
              "Each":15.084248927038628
           },
           "Topical (weight - each)":{
              "Each":6.851502702702704
           },
           "Vape Cartridge (volume - each)":{
              "Each":19.749874823014572
           },
           "Vape Cartridge (weight - each)":{
              "Each":16.83838680865686
           }
        }
    }
}

In [4]:
# Download packages, sales transactions, incoming / outgoing tranfers
company_incoming_transfer_packages_query = create_queries.create_company_incoming_transfer_packages_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)
company_outgoing_transfer_packages_query = create_queries.create_company_outgoing_transfer_packages_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)
company_sales_receipts_query = create_queries.create_company_sales_receipts_query(COMPANY_IDENTIFIER, SALES_TRANSACTIONS_START_DATE)
company_sales_transactions_query = create_queries.create_company_sales_transactions_query(COMPANY_IDENTIFIER, SALES_TRANSACTIONS_START_DATE)
company_inventory_packages_query = create_queries.create_company_inventory_packages_query(
    COMPANY_IDENTIFIER,
    include_quantity_zero=True,
)

company_incoming_transfer_packages_dataframe = pandas.read_sql_query(company_incoming_transfer_packages_query, engine)
company_outgoing_transfer_packages_dataframe = pandas.read_sql_query(company_outgoing_transfer_packages_query, engine)
company_sales_receipts_dataframe = pandas.read_sql_query(company_sales_receipts_query, engine)
company_sales_transactions_dataframe = pandas.read_sql_query(company_sales_transactions_query, engine)
company_inventory_packages_dataframe = pandas.read_sql_query(company_inventory_packages_query, engine)

In [31]:
sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts/analysis")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util
from bespoke.inventory.analysis import inventory_cogs_util as cogs_util

def _reload_libs():
    import importlib
    importlib.reload(util)
    importlib.reload(valuations_util)
    importlib.reload(cogs_util)


In [32]:
sql_helper = util.BigQuerySQLHelper(engine)

d = util.Download()
d.download_dataframes(
    incoming_transfer_packages_dataframe=company_incoming_transfer_packages_dataframe,
    outgoing_transfer_packages_dataframe=company_outgoing_transfer_packages_dataframe,
    sales_receipts_dataframe=company_sales_receipts_dataframe,
    sales_transactions_dataframe=company_sales_transactions_dataframe,
    inventory_packages_dataframe=company_inventory_packages_dataframe,
    sql_helper=sql_helper,
)

In [7]:
TODAY_DATE = date.today().strftime('%Y-%m-%d')
print(f'Today is {TODAY_DATE}')

Today is 2021-11-10


In [10]:
company_incoming_transfer_packages_dataframe['created_month'] = pandas.to_datetime(company_incoming_transfer_packages_dataframe['created_date']).dt.strftime('%Y-%m')
unique_incoming_transfer_package_months = company_incoming_transfer_packages_dataframe['created_month'].unique()

In [11]:
company_sales_receipts_dataframe['sales_month'] = pandas.to_datetime(company_sales_receipts_dataframe['sales_datetime']).dt.strftime('%Y-%m')
unique_company_sales_receipt_months = company_sales_receipts_dataframe['sales_month'].unique()

In [12]:
aggregate_unique_months = []
for month in unique_incoming_transfer_package_months:
    if month not in aggregate_unique_months:
        aggregate_unique_months.append(month)
for month in unique_company_sales_receipt_months:
    if month not in aggregate_unique_months:
        aggregate_unique_months.append(month)
aggregate_unique_months.sort()

unique_inventory_dates = []
for month in aggregate_unique_months:
    date_object = datetime.strptime(month, '%Y-%m')
    date_object = date_object.replace(day = calendar.monthrange(date_object.year, date_object.month)[1])
    eom_date_str = datetime.strftime(date_object, '%Y-%m-%d')
    if eom_date_str < TODAY_DATE:
        unique_inventory_dates.append(eom_date_str)

unique_inventory_dates.append(TODAY_DATE)
unique_inventory_dates = [datetime.strftime(datetime.strptime(unique_inventory_date, '%Y-%m-%d'), '%m/%d/%Y') for unique_inventory_date in unique_inventory_dates]

INVENTORY_DATES = unique_inventory_dates

In [34]:
q = util.Query()
q.inventory_dates = INVENTORY_DATES
q.company_name = COMPANY_NAME

id_to_history = util.get_histories(d, params=ANALYSIS_PARAMS)
util.print_counts(id_to_history)

WARN: Could not find Kief in the external pricing data config
WARN: Could not find Kief in the external pricing data config
WARN: Could not find grams in the external pricing table for category Shake
WARN: Could not find Kief in the external pricing data config
Only outgoing: 1656
Only incoming: 2067
Sold packages missing pricing information: 4 (0.03% of packages)
In and out: 1439
In and sold at least once 9780
In and sold many times 9422

 Num parent packages: 0
 num matched child packages: 547
Total pkgs: 13956


{'only_outgoing': 1656,
 'only_incoming': 2067,
 'only_sold': 4,
 'outgoing_and_incoming': 1439,
 'in_and_sold_at_least_once': 9780,
 'in_and_sold_many_times': 9422,
 'num_parent_packages': 0,
 'num_child_packages': 547,
 'total_seen': 13956}

In [35]:
date_to_inventory_packages_dataframe = {}
inventory_valuations = []

for inventory_date in INVENTORY_DATES:
    computed_inventory_package_records = util.create_inventory_dataframe_by_date(
        id_to_history,
        inventory_date,
        params=ANALYSIS_PARAMS,
    )    
    computed_inventory_packages_dataframe = pandas.DataFrame(
        computed_inventory_package_records,
        columns=util.get_inventory_column_names(),
    )
    date_to_inventory_packages_dataframe[inventory_date] = computed_inventory_packages_dataframe
    inventory_valuations.append(valuations_util.get_total_valuation_for_date(
        computed_inventory_packages_dataframe=computed_inventory_packages_dataframe,
        company_incoming_transfer_packages_dataframe=company_incoming_transfer_packages_dataframe,
    ))

WARN: incoming package #14525326 does not have a quantity
# packages in inventory: 461
valuation cost: 76717.90877192984
# packages in inventory: 886
valuation cost: 167853.21667810832
# packages in inventory: 965
valuation cost: 205729.34026506476
# packages in inventory: 556
valuation cost: 70829.15996659141
# packages in inventory: 687
valuation cost: 75629.76191464331
# packages in inventory: 816
valuation cost: 109434.38090165626
# packages in inventory: 1016
valuation cost: 137709.69042058184
# packages in inventory: 808
valuation cost: 99813.9997863248
# packages in inventory: 761
valuation cost: 102148.25999999983
# packages in inventory: 674
valuation cost: 102465.81458333333
# packages in inventory: 646
valuation cost: 85825.92416666663
# packages in inventory: 745
valuation cost: 102175.31124999998
# packages in inventory: 786
valuation cost: 87903.6629166667
# packages in inventory: 844
valuation cost: 109013.9712499999
# packages in inventory: 1157
valuation cost: 106795.3

In [36]:
from_packages_inventory_dataframe = company_inventory_packages_dataframe[[
    'package_id',
    'packaged_date',
    'unit_of_measure',
    'product_category_name',
    'product_name',
    'quantity',
]].sort_values('package_id')

package_id_to_actual_row = {}
for index, row in from_packages_inventory_dataframe.iterrows():
    package_id_to_actual_row[str(row['package_id'])] = row

res = util.compare_inventory_dataframes(
    computed=date_to_inventory_packages_dataframe[unique_inventory_dates[-1]],
    actual=from_packages_inventory_dataframe,
    options={
        'num_errors_to_show': 50,
        'accept_computed_when_sold_out': True
    }
)

Pct of # inventory matching: 98.47% (1160 / 1178)
Accuracy of quantities: 60.12%
Pct of # inventory packages over-estimated: 1.78%
Pct of # quantity over-estimated: 0.01%
Avg quantity delta: 19.21
Avg quantity: 48.17

Num matching packages: 1160
Num actual packages not computed: 18
  but computed at some point: 0, e.g., 0.00% of non-computed packages
  avg quantity from actual packages 0.00
Num computed packages not in actual: 21
  but in actual inventory at some point: 2

Computed has these extra package IDs; first 50
17567664: computed quantity 180.0 (Each)
17566994: computed quantity 180.0 (Each)
11691650: computed quantity 80.0 (Each)
19203278: computed quantity 50.0 (Each)
18573023: computed quantity 32.0 (Each)
19203285: computed quantity 32.0 (Each)
19208960: computed quantity 32.0 (Each)
19492803: computed quantity 20.0 (Each)
13740035: computed quantity 11.0 (Each)
2830324: computed quantity 8.0 (Each)
13645249: computed quantity 8.0 (Each)
2830323: computed quantity 7.0 (Each

In [ ]:
print(f'Plotting sales revenue vs cost-based inventory valuation for dates: {unique_inventory_dates}')
valuations_util.plot_inventory_and_revenue(
    q=q,
    sales_receipts_dataframe=d.sales_receipts_dataframe,
    inventory_valuations=inventory_valuations,
)

In [22]:
# For debugging individual package histories
# You have to run the above block to reload the package_id_to_history array
_reload_libs()

PACKAGE_IDS = [
  '19372996',
]

util.analyze_specific_package_histories(
    d, package_id_to_actual_row, PACKAGE_IDS, params=ANALYSIS_PARAMS)

WARN: Could not find Kief in the external pricing data config
WARN: Could not find Kief in the external pricing data config
WARN: Could not find grams in the external pricing table for category Shake
WARN: Could not find Kief in the external pricing data config
DEBUGGING PACKAGE_ID=19372996
Matching metrc_package:
package_id                     19372996
packaged_date                2021-11-04
unit_of_measure                  Pounds
product_category_name            Flower
product_name             Mai Tai Flower
quantity                         5.2489
Name: 104, dtype: object

INCOMING
{'delivery_type': 'INCOMING_INTERNAL', 'license_number': 'C11-0000020-LIC', 'manifest_number': '0002671256', 'created_date': datetime.date(2021, 11, 5), 'received_datetime': datetime.datetime(2021, 11, 5, 18, 12, 55, tzinfo=<UTC>), 'shipper_facility_license_number': 'C12-0000328-LIC', 'shipper_facility_name': 'SUNSHINE HOLISTIC', 'recipient_facility_license_number': 'C11-0000020-LIC', 'recipient_facility_n